<a href="https://colab.research.google.com/github/yuchenjohnlin/Mnist-on-EvaDB-and-DeepLake/blob/main/DeepLake_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython.display import clear_output
!pip3 install deeplake
clear_output()

#!pip install urllib3==1.26.7

import deeplake

In [2]:
from PIL import Image
import numpy as np
import os, time
import torch
from torchvision import transforms, models

##Fail data loading(Skip This part)

## Download the mnist dataset manually - Skip
1. Create a dataset folder in directory
2. download the dataset to folder
3. unzip files

Problem :

the decompressed file are not images and I don't know how to use it

In [ ]:
#!wget "https://www.kaggle.com/datasets/scolianni/mnistasjpg/data/trainingSet.tar.gz"

In [ ]:
dataset_folder = '/content/mnist_dataset'
# make sure we are in the content folder
# cwd = os.getcwd()
# if cwd!='/content' :
#   os.chdir('/content')
# cwd = os.getcwd()
# print("Current working directory:", cwd)

if not os.path.exists(dataset_folder):
    os.mkdir(dataset_folder)
    print(f"Directory '{dataset_folder}' created successfully.")
else:
    print(f"Directory '{dataset_folder}' already exists.")
os.chdir(dataset_folder)
%rm *
cwd = os.getcwd()
print("Current working directory:", cwd)

files_and_folders = os.listdir()
print(files_and_folders)
# Filter out only the directories from the list
# folders = [folder for folder in files_and_folders if os.path.isdir(folder)]
# files = [file for file in files_and_folders if os.path.isdir(file)]
# Print the list of directories
# print("Folders in the current directory:")
# print(folders)
# print("Files in the current directory:")
# print(files)


Directory '/content/mnist_dataset' already exists.
Current working directory: /content/mnist_dataset
[]


In [ ]:
data_url = "http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz"
label_url = "http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz"

data_file = os.path.basename(data_url)
label_file = os.path.basename(label_url)
if not os.path.exists(os.path.join(dataset_folder, data_file)):
    # If the file does not exist, download it using wget
    !wget -P {dataset_folder} {data_url}
    print("File downloaded successfully.")
else:
    print("File already exists. Skipping the download.")

if not os.path.exists(os.path.join(dataset_folder, label_file)):
    # If the file does not exist, download it using wget
    !wget -P {dataset_folder} {label_url}
    print("File downloaded successfully.")
else:
    print("File already exists. Skipping the download.")

print("Downloaded files in the directory:")
dir_files = os.listdir()
print(dir_files)

--2023-10-17 07:26:32--  http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
Resolving yann.lecun.com (yann.lecun.com)... 104.21.29.36, 172.67.171.76, 2606:4700:3034::6815:1d24, ...
Connecting to yann.lecun.com (yann.lecun.com)|104.21.29.36|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1648877 (1.6M) [application/x-gzip]
Saving to: ‘/content/mnist_dataset/t10k-images-idx3-ubyte.gz’

t10k-images-idx3-ub 100%[===================>]   1.57M  --.-KB/s    in 0.04s   

2023-10-17 07:26:33 (35.3 MB/s) - ‘/content/mnist_dataset/t10k-images-idx3-ubyte.gz’ saved [1648877/1648877]

File downloaded successfully.
--2023-10-17 07:26:33--  http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz
Resolving yann.lecun.com (yann.lecun.com)... 104.21.29.36, 172.67.171.76, 2606:4700:3036::ac43:ab4c, ...
Connecting to yann.lecun.com (yann.lecun.com)|104.21.29.36|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4542 (4.4K) [application/x-gzip]
Savin

2. unzip the files

In [ ]:

import gzip
import shutil

# Filter for files with .gz extension
gz_files = [file for file in dir_files if file.endswith('.gz')]

for gzfile in gz_files:
  print(gzfile)
  #!tar -xzvf {gzfile}

# Path to the input .gz file
input_gz_file = os.path.join(dataset_folder,gz_files[0])
print(type(input_gz_file))

# Path to the output uncompressed file (without .gz extension)
output_file = 'image'
output_path = os.path.join(dataset_folder,output_file)

# Open the .gz file and the output file, then decompress and write the data
with gzip.open(input_gz_file, 'rb') as f_in, open(output_path, 'wb') as f_out:
    shutil.copyfileobj(f_in, f_out)

print(f'File {input_gz_file} has been successfully decompressed to {output_path}.')
print(os.listdir())


t10k-images-idx3-ubyte.gz
t10k-labels-idx1-ubyte.gz
<class 'str'>
File /content/mnist_dataset/t10k-images-idx3-ubyte.gz has been successfully decompressed to /content/mnist_dataset/image.
['t10k-images-idx3-ubyte.gz', 't10k-labels-idx1-ubyte.gz', 'image']


##Another dataset - Skip

In [ ]:
from keras.datasets import mnist
(train_X, train_y), (test_X, test_y) = mnist.load_data()

11490434/11490434 [==============================] - 1s 0us/step


##Dataset from TorchVision - Skip

Try putting this into deeplake -> but I didn't find any way to import PIL images into deeplake

Can refer to this [website](https://docs.deeplake.ai/en/latest/Compressions.html) about the compression and data types that can be stored in the deeplake dataset

However,
Deeplake already has the mnist data set which can be seen [here](https://app.activeloop.ai/activeloop/mnist-test/)

In [ ]:
import torch
import os
print(torch.__version__)
import torchvision
import torchvision.datasets as datasets
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=None)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=None)
print("Length of trainset : ",len(mnist_trainset))
print("Length of test set : ",len(mnist_testset))
front_data, back_data = mnist_trainset[6]
print("Type of data in train dataset : ", type(mnist_trainset[0]), type(front_data), type(back_data))
display(front_data)
print(back_data)

os.chdir('data')
print(os.listdir())


2.0.1+cu118


100%|██████████| 9912422/9912422 [00:00<00:00, 74711288.25it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 33361524.05it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 19596978.79it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 14586928.61it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Length of trainset :  60000
Length of test set :  10000
Type of data in train dataset :  <class 'tuple'> <class 'PIL.Image.Image'> <class 'int'>


1
['MNIST', 'data']
['MNIST']


##Start Here

# Load data
1. Download the mnist jpg dataset from kaggle
2. Extract the files
3. Upload the jpgs to deeplake dataset


In [3]:
!pip install opendatasets
import opendatasets as od
od.download("https://www.kaggle.com/datasets/scolianni/mnistasjpg/data")
clear_output()
print(os.listdir())


['.config', 'mnistasjpg', 'animals_deeplake', 'mnist_jpg', 'animals']


In [4]:
# Create dataset in deeplake format
deep_lake_path = "hub://organization_name/dataset_name"

ds = deeplake.empty('./mnist_jpg', overwrite=True) # Create the dataset locally

# Find the class_names and list of files that need to be uploaded
dataset_folder = './mnistasjpg'
cwd = os.getcwd()
print("Current working directory:", cwd)

# Find the subfolders, but filter additional files like DS_Store that are added on Mac machines.
set_names = [item for item in os.listdir(dataset_folder) if os.path.isdir(os.path.join(dataset_folder, item))]
print(set_names)

# gets the label list from the training sett filders
#
# !!!!! because the order of the trainingSet list is different from Training Sample so be careful
#

trainset_folder = os.path.join(dataset_folder,'trainingSet/trainingSet')
class_names = [item for item in os.listdir(trainset_folder) if os.path.isdir(os.path.join(trainset_folder, item))]
print(class_names)

files_list = []

# Get only files in trainingSet

for dirpath, dirnames, filenames in os.walk(dataset_folder):
  print("Dirpath : ", dirpath)
  #print("Dirname : ", dirnames)
  if dirpath.startswith(os.path.join(dataset_folder, 'trainingSet')) and os.path.basename(dirpath) in ['3','1','2','4','5','6','7','8','9']: #,'testSet', 'testSample']:
    for filename in filenames:
      #print("filename : ", filename)
      files_list.append(os.path.join(dirpath, filename))
#print(files_list)
print(len(files_list))



./mnist_jpg loaded successfully.
Current working directory: /content
['testSet', 'testSample', 'trainingSample', 'trainingSet']
['3', '0', '5', '4', '7', '8', '1', '6', '2', '9']
Dirpath :  ./mnistasjpg
Dirpath :  ./mnistasjpg/testSet
Dirpath :  ./mnistasjpg/testSet/testSet
Dirpath :  ./mnistasjpg/testSample
Dirpath :  ./mnistasjpg/testSample/testSample
Dirpath :  ./mnistasjpg/trainingSample
Dirpath :  ./mnistasjpg/trainingSample/trainingSample
Dirpath :  ./mnistasjpg/trainingSample/trainingSample/3
Dirpath :  ./mnistasjpg/trainingSample/trainingSample/0
Dirpath :  ./mnistasjpg/trainingSample/trainingSample/5
Dirpath :  ./mnistasjpg/trainingSample/trainingSample/4
Dirpath :  ./mnistasjpg/trainingSample/trainingSample/7
Dirpath :  ./mnistasjpg/trainingSample/trainingSample/8
Dirpath :  ./mnistasjpg/trainingSample/trainingSample/1
Dirpath :  ./mnistasjpg/trainingSample/trainingSample/6
Dirpath :  ./mnistasjpg/trainingSample/trainingSample/2
Dirpath :  ./mnistasjpg/trainingSample/training

Dirpath :  ./mnistasjpg/trainingSet/trainingSet/0
Dirpath :  ./mnistasjpg/trainingSet/trainingSet/5
Dirpath :  ./mnistasjpg/trainingSet/trainingSet/4
Dirpath :  ./mnistasjpg/trainingSet/trainingSet/7
Dirpath :  ./mnistasjpg/trainingSet/trainingSet/8
Dirpath :  ./mnistasjpg/trainingSet/trainingSet/1
Dirpath :  ./mnistasjpg/trainingSet/trainingSet/6
Dirpath :  ./mnistasjpg/trainingSet/trainingSet/2
Dirpath :  ./mnistasjpg/trainingSet/trainingSet/9
37868


Create the dataset tensors and Upload metadata

In [5]:

with ds:
    # Create the tensors with names of your choice.
    ds.create_tensor('images', htype = 'image', sample_compression = 'jpeg', exist_ok=True)
    ds.create_tensor('labels', htype = 'class_label', class_names = class_names, exist_ok=True)

    # Add arbitrary metadata - Optional
    #ds.info.update(description = 'My first Deep Lake dataset')
    #ds.images.info.update(camera_type = 'SLR')


populate the data in the tensors

In [6]:
load_data2_start = time.time()
with ds:
    # Iterate through the files and append to Deep Lake dataset
    for file in files_list:
        label_text = os.path.basename(os.path.dirname(file))
        #print(label_text)
        label_num = class_names.index(label_text)

        #Append data to the tensors
        ds.append({'images': deeplake.read(file), 'labels': np.uint32(label_num)})

  # Warning !!! :  Grayscale images will be reshaped from (H, W) to (H, W, 1) to match tensor dimensions.
load_data2_end = time.time()

data_load_time = load_data2_end - load_data2_start
print(data_load_time)

/usr/local/lib/python3.10/dist-packages/deeplake/core/chunk/base_chunk.py:421: UserWarning: Grayscale images will be reshaped from (H, W) to (H, W, 1) to match tensor dimensions. This warning will be shown only once.
  warnings.warn(message)


83.73714447021484


Problem : the image that is stored into the deeplake dataset cannot be recognized after transforming into a PIL image

The reason is the shape is (28,28,1) I think because there is only 1 dimension on the 0 axis because it is a gray scale image

So we have to squeeze it to a (28,28) so that it can recognize

In [7]:
#print(type(ds.images[0].numpy()))
#print(ds.images[0].numpy())
im0 = ds.images[0].numpy()
### !!!!!!!!!!!!!!!!!!!!!!!!!

im0 = np.squeeze(im0, axis=2)

### !!!!!!!!!!!!!!!!!!!!!!!!!
pil0 = Image.fromarray(im0)
display(pil0)

label0 = ds.labels[0].numpy()
print(label0)
#Image.fromarray(ds.images[0].numpy())
ds.summary()


[0]
Dataset(path='./mnist_jpg', tensors=['images', 'labels'])

 tensor      htype           shape          dtype  compression
 -------    -------         -------        -------  ------- 
 images      image     (37868, 28, 28, 1)   uint8    jpeg   
 labels   class_label      (37868, 1)      uint32    None   


#Test loading - Skip (just for verification)
I manually uploaded a few jpg files to test its functionality

It is normal if it doesn't work

In [ ]:
%cd /content
# Find the class_names and list of files that need to be uploaded
dataset_folder = './animals'

ds_animal = deeplake.empty('./animals_deeplake', overwrite=True)

# Find the subfolders, but filter additional files like DS_Store that are added on Mac machines.
class_names = [item for item in os.listdir(dataset_folder) if os.path.isdir(os.path.join(dataset_folder, item))]

files_list = []
for dirpath, dirnames, filenames in os.walk(dataset_folder):
    for filename in filenames:
        files_list.append(os.path.join(dirpath, filename))

with ds_animal:
    # Create the tensors with names of your choice.
    ds_animal.create_tensor('images', htype = 'image', sample_compression = 'jpeg', exist_ok=True)
    ds_animal.create_tensor('labels', htype = 'class_label', class_names = class_names , exist_ok=True)

    # Add arbitrary metadata - Optional
    ds_animal.info.update(description = 'My first Deep Lake dataset')
    ds_animal.images.info.update(camera_type = 'SLR')

with ds_animal:
    # Iterate through the files and append to Deep Lake dataset
    for file in files_list:
        label_text = os.path.basename(os.path.dirname(file))
        label_num = class_names.index(label_text)

        #Append data to the tensors
        ds_animal.append({'images': deeplake.read(file), 'labels': np.uint32(label_num)})

/content
./animals_deeplake loaded successfully.


In [ ]:
#print(type(ds_animal.images[0]))
print(ds_animal.images[0].numpy().shape)
print(ds_animal.labels[0].numpy())
#print(ds_animal.images[0].numpy())

im = Image.fromarray(ds_animal.images[0].numpy())
print(type(im))
ds_animal.summary()

(960, 640, 3)
[0]
<class 'PIL.Image.Image'>
Dataset(path='./animals_deeplake', tensors=['images', 'labels'])

 tensor      htype             shape           dtype  compression
 -------    -------           -------         -------  ------- 
 images      image     (4, 427:1026, 640, 3)   uint8    jpeg   
 labels   class_label         (4, 1)          uint32    None   


##Deeplake builtin Mnist dataset
1. Use builtin dataloader to load the data
2. To make the comparison accurate, use the model specified in EvaDB
3. Perform model testing using the test data set

#Load Data

In [ ]:
# Connect to the training and testing datasets
ds_train = deeplake.load('hub://activeloop/mnist-train')
ds_test = deeplake.load('hub://activeloop/mnist-test')

/

Opening dataset in read-only mode as you don't have write permissions.


\

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/mnist-train



\

...

Opening dataset in read-only mode as you don't have write permissions.


|

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/mnist-test



|

hub://activeloop/mnist-test loaded successfully.



Print data type and data stored using the dataloader

We can see the image is still in PIL but the dataloader is different from on in pytorch

In [ ]:
display(ds_train)
print("Type of data in deeplake dataset : ", type(ds_train[0]))

print(ds_train.images[0].numpy().shape)
print(ds_train.labels[0].numpy().shape)

img = ds_train.images[0].numpy()
print("Type of image in deeplake dataset : ", type(img))
img.shape
label = ds_train.labels[0].numpy()
print("Type of label in deeplake dataset : ", type(label))
label.shape
display(Image.fromarray(ds_train.images[0].numpy(), "L"))
print(label)


Dataset(path='hub://activeloop/mnist-train', read_only=True, tensors=['images', 'labels'])

Type of data in deeplake dataset :  <class 'deeplake.core.dataset.deeplake_cloud_dataset.DeepLakeCloudDataset'>
(28, 28)
(1,)
Type of image in deeplake dataset :  <class 'numpy.ndarray'>
Type of label in deeplake dataset :  <class 'numpy.ndarray'>


[5]


#Mnist Model

Notes :  

Try to use the model that was provided in evadb [github](https://github.com/aaron-xichen/pytorch-playground/blob/master/utee/selector.py).

The problem was that it used the selector to activate the mnist.py but because the hardware he was using had GPUs or CUDAs ? so there will be errors for me.

But it occurred to me that I could run the mnist in Eva DB so I think using the one in Evadb will work.

Specify Device

In [8]:
# this was from the deeplake code example
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cpu


In [9]:
import torch.nn as nn
from collections import OrderedDict
import torch.utils.model_zoo as model_zoo

model_urls = {
    'mnist': 'http://ml.cs.tsinghua.edu.cn/~chenxi/pytorch-models/mnist-b07bb66b.pth'
}

In [10]:
# here EvaDB used a class name called MnistImageClassifier - I think I will have to know how they call this function or
# how they run the model on there data using sql
class MLP(nn.Module):
  def __init__(self, input_dims, n_hiddens, n_class):
    super(MLP, self).__init__()
    assert isinstance(input_dims, int), 'Please provide int for input_dims'
    self.input_dims = input_dims
    current_dims = input_dims
    layers = OrderedDict()

    if isinstance(n_hiddens, int):
        n_hiddens = [n_hiddens]
    else:
        n_hiddens = list(n_hiddens)
    for i, n_hidden in enumerate(n_hiddens):
        layers['fc{}'.format(i+1)] = nn.Linear(current_dims, n_hidden)
        layers['relu{}'.format(i+1)] = nn.ReLU()
        layers['drop{}'.format(i+1)] = nn.Dropout(0.2)
        current_dims = n_hidden
    layers['out'] = nn.Linear(current_dims, n_class)

    self.model= nn.Sequential(layers)
    print(self.model)

  def forward(self, input):
    input = input.view(input.size(0), -1)
    assert input.size(1) == self.input_dims
    return self.model.forward(input)

def mnist(input_dims=784, n_hiddens=[256, 256], n_class=10, pretrained=None):
  model = MLP(input_dims, n_hiddens, n_class)
  if pretrained is not None:
      m = model_zoo.load_url(model_urls['mnist'],
                             map_location=torch.device(device)) # this is the line where EvaDB was different from the github code
      state_dict = m.state_dict() if isinstance(m, nn.Module) else m
      assert isinstance(state_dict, (dict, OrderedDict)), type(state_dict)
      model.load_state_dict(state_dict)
  return model

# EvaDB uses other functions in their self-defined Class to do transform and turning data to panda



Test function

In [11]:
import matplotlib.pyplot as plt

def test_model(model, data_loader, device):
  model.eval()

  start_time = time.time()
  total = 0;
  correct = 0
  with torch.no_grad(): # this deactivates the gradient computation which is not needed for model evaluation and inference, turning this off can make things run faster , this is mostky for backpropagation while training
    for i, data in enumerate(data_loader): # here we use the data in the data_loader , but because it is not transformed so I don;t know if it will work
      inputs = data['images']
      labels = torch.squeeze(data['labels'])

      #print(inputs)

      inputs = inputs.to(device)
      labels = labels.to(device)

      # chatgpt said that this line is not needed and should be removed
      #optimizer.zero_grad()

      # chatgpt says that this only does forward pass
      outputs = model(inputs.float())

      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      #print("Predicted : ", predicted)
      correct += (predicted == labels).sum().item()
      accuracy = 100 * correct / total

      #outcome = []
      #predictions = model(inputs)
      #for prediction in predictions:
      #  label = model.as_numpy(prediction.data.argmax())
      #  outcome.append({"label": str(label)})

      # Print predictions, actual labels, and images
      # for j in range(inputs.size(0)):
      #     print('Prediction: {}, Actual Label: {}'.format(predicted[j].item(), labels[j].item()))
      #     #Assuming inputs are PIL images, you can use matplotlib to display the images
      #     plt.imshow(inputs[j].permute(1, 2, 0).cpu().numpy())  # Assuming channels-last format
      #     plt.show()

      #     image_np = inputs[j].permute(1, 2, 0).cpu().numpy()
      #     image_2d = image_np[:, :, 0]
      #     display(Image.fromarray(image_2d, "L"))
      #print(i)
      # if i%15 == 0:
      #   print('Prediction: {}, Actual Label: {}'.format(predicted[0].item(), labels[0].item()))
      #   #Assuming inputs are PIL images, you can use matplotlib to display the images
      #   plt.imshow(inputs[0].permute(1, 2, 0).cpu().numpy())  # Assuming channels-last format
      #   plt.show()
      #   # this printing doesn't work I guess
      #   image_np = inputs[0].permute(1, 2, 0).cpu().numpy()
      #   image_2d = image_np[:, :, 0]
      #   display(Image.fromarray(image_2d, "L"))

      print('Finished Testing')
      #print('Total images: {}, Correct: {}'.format(total, correct))
      print('Testing accuracy: %.1f %%' %(accuracy))


In [12]:
def get_pred_model(model, data_loader, device, pred_num):
  model.eval()

  result = []
  with torch.no_grad(): # this deactivates the gradient computation which is not needed for model evaluation and inference, turning this off can make things run faster , this is mostky for backpropagation while training
    for i, data in enumerate(data_loader): # here we use the data in the data_loader , but because it is not transformed so I don;t know if it will work
      inputs = data['images']
      labels = torch.squeeze(data['labels'])

      inputs = inputs.to(device)
      labels = labels.to(device)

      # chatgpt says that this only does forward pass
      outputs = model(inputs.float())
      _, predicted = torch.max(outputs.data, 1)
      for j in range(inputs.size(0)):
        if predicted[j].item()==pred_num:
          result.append(inputs[j])

  return result

In [13]:
# why do we need a transform here?
# This was the transform function for pretrained ResNet18 neural network from torchvision.models
'''
tform = transforms.Compose([
   transforms.RandomRotation(20), # Image augmentation
   transforms.ToTensor(), # Must convert to pytorch tensor for subsequent operations to run
   transforms.Normalize([0.5], [0.5]),
])
'''

tform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

#Test model
Currently there are bugs when doing the test because the labels are incorrectly stored into the dataset, but the predictions are right

In [15]:
batch_size = 100

# Since torchvision transforms expect PIL images, we use the 'pil' decode_method for the 'images' tensor. This is much faster than running ToPILImage inside the transform

#train_loader = ds_train.pytorch(num_workers = 0, shuffle = True, transform = {'images': tform, 'labels': None}, batch_size = batch_size, decode_method = {'images': 'pil'})
#test_loader = ds_test.pytorch(num_workers = 0, transform = {'images': tform, 'labels': None}, batch_size = batch_size, decode_method = {'images': 'pil'})

#hope this works
mnist_loader = ds.pytorch(num_workers = 0, transform = {'images': tform, 'labels': None}, batch_size = batch_size, decode_method = {'images': 'pil'})


In [17]:
model = mnist(pretrained=True)
#test_model(model, test_loader, device)

#print("Type of model is : ", type(model))
runall_start = time.time()
test_model(model, mnist_loader, device)
runall_end = time.time()

runall_time = runall_end - runall_start
print(f"Run all data time : {runall_time} seconds")

pred_num = 6
query_start = time.time()

get_label6 = get_pred_model(model, mnist_loader, device, pred_num)
# print(type(get_label6[0]))
# for i in range(0,len(get_label6)-1) :
#   plt.imshow(get_label6[i].permute(1, 2, 0).cpu().numpy())  # Assuming channels-last format
#   plt.show()

query_end = time.time()
query_time = query_end - query_start
print(f"Query data time : {query_time} seconds")

#this can print out the plots of results





Sequential(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (relu1): ReLU()
  (drop1): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (relu2): ReLU()
  (drop2): Dropout(p=0.2, inplace=False)
  (out): Linear(in_features=256, out_features=10, bias=True)
)
Finished Testing
Testing accuracy: 0.0 %
Finished Testing
Testing accuracy: 0.0 %
Finished Testing
Testing accuracy: 0.0 %
Finished Testing
Testing accuracy: 0.0 %
Finished Testing
Testing accuracy: 0.0 %
Finished Testing
Testing accuracy: 0.0 %
Finished Testing
Testing accuracy: 0.0 %
Finished Testing
Testing accuracy: 0.0 %
Finished Testing
Testing accuracy: 0.0 %
Finished Testing
Testing accuracy: 0.0 %
Finished Testing
Testing accuracy: 0.0 %
Finished Testing
Testing accuracy: 0.0 %
Finished Testing
Testing accuracy: 0.0 %
Finished Testing
Testing accuracy: 0.0 %
Finished Testing
Testing accuracy: 0.0 %
Finished Testing
Testing accuracy: 0.0 %
Finished Testing
Testing ac

#Run Query - Still working
In deeplake they call it Tensor query maybe because the data stored in the database are Tensors

In [ ]:
query = f"select images where labels==6"
result = ds.query(query)

EmptyTokenException: ignored

#End

If we want to use the pytorch function of the dataset of the function deeplake.load then we have to store the data into deeplake
Deeplake provides a pytorch dataloader that connects the Deep Lake dataset to the PyTorch model using the provided method ds.pytorch().

This method automatically applies the transformation function and takes care of random shuffling (if desired). The num_workers parameter can be used to parallelize data preprocessing, which is critical for ensuring that preprocessing does not bottleneck the overall training workflow.
The transform input is a dictionary where the key is the tensor name and the value is the transformation function that should be applied to that tensor. If a specific tensor's data does not need to be returned, it should be omitted from the keys. If a tensor's data does not need to be modified during preprocessing, the transformation function is set as None, which will convert the input data to a torch tensor.